In [78]:
#Generates a .csv file from data from https://simplemaps.com/data/gb-cities
#Note - this code is specific to generating a .csv list of cities from this particular format of data.
import csv
file = open("gb.csv")
reader = csv.reader(file)
lines = len(list(reader))
file.close()
file = open("List of cities.csv", "w")
file.close()
for row in range(1,lines-1):
    file = open("gb.csv")
    reader = csv.reader(file)
    cityName = list(reader)[row][0]
    file.close()
    file = open("List of cities.csv", "a")
    file.write(cityName + ",")
    file.close()

file = open("gb.csv")
reader = csv.reader(file)
cityName = list(reader)[lines-1][0]
file.close()
file = open("List of cities.csv", "a")
file.write(cityName)
file.close()

In [136]:
import csv
file = open("List of cities.csv", "r")
reader = csv.reader(file)
l = list(reader) #creates an array of cities
file.close()

file = open("failedQueries.txt", "w") #deletes file if already exists
file.close()

import requests
for city in range(0,1000):
    cityName = l[0][city]  #As it is from a .csv file, the array is 2D, i.e. [[London,Birmingham...,Broughton]] - hence 2 indexes are required
    api_url = "https://nominatim.openstreetmap.org/search.php?city=" + cityName + "&country=United+Kingdom&polygon_geojson=1&format=json"
    response = requests.get(api_url) #retrieves geoJSON file from Nominatim API
    geoJ = str(response.json())
    tempGeoJ = geoJ #stores the geoJ string in a temporary variable which is changed throughout the loop
    fail = True
    while True:
        start = tempGeoJ.find("{'type': 'Polygon', 'coordinates': [[") #This substring depends on the format of the geoJ file! e.g. "" may be used instead of ''
        if start == -1:
            start = tempGeoJ.find("{'type': 'MultiPolygon', 'coordinates': [[")
        end = tempGeoJ.find("]]}")
        if start == -1: 
            if fail == True: 
                file = open("failedQueries.txt", "a")
                file.write(cityName)
                file.write(geoJ)
                file.write("\n")
                file.close()
            break
        
        cityName2 = ""
        nameStart = tempGeoJ.find("display_name") + 16
        char = tempGeoJ[nameStart] 
        while char != "'":
            if char != "/":                
                cityName2 = cityName2 + char
            nameStart += 1
            char = tempGeoJ[nameStart] #This finds the name of the city, which the file is named after
        
        
        parsedGeoJ = tempGeoJ[start:end+3] #creates parsed geoJ file
        file = open("C:/Users/Toan/Documents/CityGeoJSON/United Kingdom/GeoJSON files/" + cityName2 + ".json", "w") #already exists with same name?
        file.write(parsedGeoJ)
        file.close()
        tempGeoJ = tempGeoJ[end+3:]
        fail = False #If the loop has executed at least once without a failure, it does not need to be logged as a failure.